In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from joblib import dump

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Calculate the index to split files into training and testing sets
split_index = int(len(csv_files) * 0.7)

# Training set and Testing set
train_files = csv_files[:split_index]
test_files = csv_files[split_index:]

# Initialize common columns with columns from the first CSV file
common_columns = set(pd.read_csv(os.path.join(folder_path, csv_files[0])).columns)

# Loop through each CSV file in the folder to find common columns
for csv_file in csv_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

# Print the common variables found in all files
print("\nCommon Variables:")
print(common_columns)

# Initialize an empty set to store all unique columns
all_columns = set()

# Loop through each CSV file in the folder to find all unique columns
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

# Find the uncommon variables
uncommon_variables = all_columns - common_columns

# Print the uncommon variables
print("\nUncommon Variables:")
print(uncommon_variables)

# Initialize an empty list to store the files in memory
files_in_memory = []

# Initialize the model
model = CatBoostRegressor(iterations=1000, learning_rate=0.01, random_state=42, verbose=True)

# Loop through each CSV file in the folder
for idx, csv_file in enumerate(csv_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]
    
    # Concatenate the data to the DataFrame
    if idx == 0:
        concatenated_data = df
    else:
        concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
        y = y.values.reshape(-1)
        y = y[:len(X)]
        #y = y.iloc[:, 0]

    # Fit the model on the new data
    model.fit(X, y)
    
    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")
    
    # Update files in memory
    files_in_memory.append(csv_file)

# Print all files in memory
print(f"All files in memory: {files_in_memory}")

# Extract features (X) and target (y) from concatenated data for test set
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

# Ensure y contains only one column
if isinstance(y_test, pd.DataFrame):
    y_test = y_test.values.reshape(-1)
    y_test = y_test[:len(X_test)]

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# Save the trained model as a .pkl file
model_filename = 'CatBoost_model.pkl'
dump(model, model_filename)
print(f"Model saved as {model_filename}")



Common Variables:
{'1715 IMG_TIR1', '0015 IMG_TIR1', '0445 IMG_WV', '1115 IMG_TIR1', '1845 IMG_WV', '0345 IMG_WV', '1445 IMG_TIR1', '0615 IMG_WV', '0315 IMG_TIR1', '0945 IMG_TIR1', '1945 IMG_TIR2', '2315 IMG_TIR2', '0045 IMG_TIR2', '2215 IMG_WV', '1645 IMG_TIR1', '1545 IMG_TIR2', '2115 IMG_TIR1', '2145 IMG_TIR1', '1515 IMG_TIR2', '2345 IMG_TIR1', '1415 IMG_TIR1', '1915 IMG_WV', 'Date', '2345 IMG_TIR2', '0315 IMG_TIR2', '0645 IMG_TIR1', '0945 IMG_TIR2', '2045 IMG_TIR2', '2115 IMG_WV', '0345 IMG_TIR2', '1015 IMG_WV', '0445 IMG_TIR1', '2045 IMG_WV', '0715 IMG_WV', '1245 IMG_TIR2', '1245 IMG_WV', '1715 IMG_TIR2', '1315 IMG_TIR1', '1915 IMG_TIR2', '1945 IMG_TIR1', '1115 IMG_TIR2', '0945 IMG_WV', '2045 IMG_TIR1', '2215 IMG_TIR2', '0715 IMG_TIR2', '1815 IMG_TIR1', '1315 IMG_WV', '1015 IMG_TIR1', '0245 IMG_TIR1', '1745 IMG_TIR1', '2145 IMG_TIR2', '0615 IMG_TIR1', '2315 IMG_WV', '1345 IMG_TIR2', '1815 IMG_WV', '1045 IMG_WV', '0745 IMG_WV', '0245 IMG_TIR2', '1815 IMG_TIR2', 'longitude', '1515 I

143:	learn: 10.6841945	total: 809ms	remaining: 4.81s
144:	learn: 10.6798863	total: 815ms	remaining: 4.81s
145:	learn: 10.6773214	total: 821ms	remaining: 4.8s
146:	learn: 10.6749480	total: 826ms	remaining: 4.79s
147:	learn: 10.6722516	total: 831ms	remaining: 4.79s
148:	learn: 10.6702036	total: 837ms	remaining: 4.78s
149:	learn: 10.6679768	total: 842ms	remaining: 4.77s
150:	learn: 10.6650289	total: 848ms	remaining: 4.77s
151:	learn: 10.6617877	total: 853ms	remaining: 4.76s
152:	learn: 10.6594612	total: 859ms	remaining: 4.75s
153:	learn: 10.6566355	total: 864ms	remaining: 4.75s
154:	learn: 10.6542478	total: 869ms	remaining: 4.74s
155:	learn: 10.6526602	total: 874ms	remaining: 4.73s
156:	learn: 10.6502236	total: 880ms	remaining: 4.72s
157:	learn: 10.6477028	total: 885ms	remaining: 4.72s
158:	learn: 10.6458890	total: 890ms	remaining: 4.71s
159:	learn: 10.6435268	total: 896ms	remaining: 4.7s
160:	learn: 10.6416537	total: 901ms	remaining: 4.69s
161:	learn: 10.6391627	total: 906ms	remaining: 4

332:	learn: 10.3259805	total: 1.82s	remaining: 3.65s
333:	learn: 10.3231184	total: 1.82s	remaining: 3.64s
334:	learn: 10.3225644	total: 1.83s	remaining: 3.64s
335:	learn: 10.3216077	total: 1.84s	remaining: 3.63s
336:	learn: 10.3207435	total: 1.84s	remaining: 3.63s
337:	learn: 10.3198589	total: 1.85s	remaining: 3.62s
338:	learn: 10.3178802	total: 1.85s	remaining: 3.62s
339:	learn: 10.3167060	total: 1.86s	remaining: 3.61s
340:	learn: 10.3146544	total: 1.86s	remaining: 3.6s
341:	learn: 10.3123201	total: 1.87s	remaining: 3.6s
342:	learn: 10.3083833	total: 1.88s	remaining: 3.59s
343:	learn: 10.3074334	total: 1.88s	remaining: 3.59s
344:	learn: 10.3066672	total: 1.89s	remaining: 3.58s
345:	learn: 10.3057645	total: 1.89s	remaining: 3.57s
346:	learn: 10.3047091	total: 1.9s	remaining: 3.57s
347:	learn: 10.3037136	total: 1.9s	remaining: 3.56s
348:	learn: 10.3030904	total: 1.91s	remaining: 3.56s
349:	learn: 10.3015389	total: 1.91s	remaining: 3.55s
350:	learn: 10.3000350	total: 1.92s	remaining: 3.5

508:	learn: 10.0994869	total: 2.81s	remaining: 2.71s
509:	learn: 10.0977300	total: 2.83s	remaining: 2.71s
510:	learn: 10.0972510	total: 2.83s	remaining: 2.71s
511:	learn: 10.0962258	total: 2.84s	remaining: 2.71s
512:	learn: 10.0950059	total: 2.85s	remaining: 2.7s
513:	learn: 10.0912633	total: 2.85s	remaining: 2.69s
514:	learn: 10.0908101	total: 2.85s	remaining: 2.69s
515:	learn: 10.0894221	total: 2.86s	remaining: 2.68s
516:	learn: 10.0884876	total: 2.87s	remaining: 2.68s
517:	learn: 10.0817414	total: 2.87s	remaining: 2.67s
518:	learn: 10.0806256	total: 2.88s	remaining: 2.67s
519:	learn: 10.0797891	total: 2.88s	remaining: 2.66s
520:	learn: 10.0788824	total: 2.89s	remaining: 2.65s
521:	learn: 10.0778047	total: 2.89s	remaining: 2.65s
522:	learn: 10.0765171	total: 2.9s	remaining: 2.64s
523:	learn: 10.0754940	total: 2.9s	remaining: 2.64s
524:	learn: 10.0745291	total: 2.91s	remaining: 2.63s
525:	learn: 10.0737842	total: 2.91s	remaining: 2.63s
526:	learn: 10.0699086	total: 2.92s	remaining: 2.

693:	learn: 9.8693925	total: 3.83s	remaining: 1.69s
694:	learn: 9.8689557	total: 3.83s	remaining: 1.68s
695:	learn: 9.8683589	total: 3.84s	remaining: 1.68s
696:	learn: 9.8665218	total: 3.84s	remaining: 1.67s
697:	learn: 9.8653573	total: 3.85s	remaining: 1.66s
698:	learn: 9.8647531	total: 3.85s	remaining: 1.66s
699:	learn: 9.8638799	total: 3.86s	remaining: 1.65s
700:	learn: 9.8627284	total: 3.86s	remaining: 1.65s
701:	learn: 9.8616169	total: 3.87s	remaining: 1.64s
702:	learn: 9.8611786	total: 3.87s	remaining: 1.64s
703:	learn: 9.8587305	total: 3.88s	remaining: 1.63s
704:	learn: 9.8577675	total: 3.88s	remaining: 1.63s
705:	learn: 9.8569138	total: 3.89s	remaining: 1.62s
706:	learn: 9.8538954	total: 3.9s	remaining: 1.61s
707:	learn: 9.8531433	total: 3.9s	remaining: 1.61s
708:	learn: 9.8521085	total: 3.9s	remaining: 1.6s
709:	learn: 9.8511212	total: 3.91s	remaining: 1.6s
710:	learn: 9.8502362	total: 3.92s	remaining: 1.59s
711:	learn: 9.8498625	total: 3.92s	remaining: 1.59s
712:	learn: 9.847

...

190:	learn: 15.0599805	total: 1.01s	remaining: 4.29s
191:	learn: 15.0560448	total: 1.02s	remaining: 4.29s
192:	learn: 15.0521529	total: 1.02s	remaining: 4.28s
193:	learn: 15.0482892	total: 1.03s	remaining: 4.28s
194:	learn: 15.0360853	total: 1.03s	remaining: 4.27s
195:	learn: 15.0237508	total: 1.04s	remaining: 4.26s
196:	learn: 15.0199724	total: 1.04s	remaining: 4.26s
197:	learn: 15.0146023	total: 1.05s	remaining: 4.25s
198:	learn: 15.0092124	total: 1.05s	remaining: 4.25s
199:	learn: 14.9999200	total: 1.06s	remaining: 4.24s
200:	learn: 14.9910180	total: 1.06s	remaining: 4.24s
201:	learn: 14.9875339	total: 1.07s	remaining: 4.23s
202:	learn: 14.9772858	total: 1.08s	remaining: 4.23s
203:	learn: 14.9639608	total: 1.08s	remaining: 4.22s
204:	learn: 14.9612469	total: 1.09s	remaining: 4.22s
205:	learn: 14.9553563	total: 1.09s	remaining: 4.21s
206:	learn: 14.9474724	total: 1.1s	remaining: 4.21s
207:	learn: 14.9444888	total: 1.1s	remaining: 4.2s
208:	learn: 14.9400609	total: 1.11s	remaining: 4.

346:	learn: 14.2404830	total: 1.82s	remaining: 3.42s
347:	learn: 14.2313163	total: 1.82s	remaining: 3.42s
348:	learn: 14.2229888	total: 1.83s	remaining: 3.41s
349:	learn: 14.2153580	total: 1.83s	remaining: 3.41s
350:	learn: 14.2127476	total: 1.84s	remaining: 3.4s
351:	learn: 14.2084031	total: 1.84s	remaining: 3.4s
352:	learn: 14.2031367	total: 1.85s	remaining: 3.39s
353:	learn: 14.1995381	total: 1.85s	remaining: 3.39s
354:	learn: 14.1981831	total: 1.86s	remaining: 3.38s
355:	learn: 14.1929534	total: 1.87s	remaining: 3.38s
356:	learn: 14.1903155	total: 1.87s	remaining: 3.37s
357:	learn: 14.1858576	total: 1.88s	remaining: 3.37s
358:	learn: 14.1790064	total: 1.88s	remaining: 3.36s
359:	learn: 14.1698164	total: 1.89s	remaining: 3.35s
360:	learn: 14.1622670	total: 1.89s	remaining: 3.35s
361:	learn: 14.1581433	total: 1.9s	remaining: 3.34s
362:	learn: 14.1555833	total: 1.9s	remaining: 3.34s
363:	learn: 14.1475943	total: 1.91s	remaining: 3.33s
364:	learn: 14.1418639	total: 1.91s	remaining: 3.3

541:	learn: 13.4002843	total: 2.83s	remaining: 2.39s
542:	learn: 13.3989745	total: 2.83s	remaining: 2.38s
543:	learn: 13.3974926	total: 2.84s	remaining: 2.38s
544:	learn: 13.3939715	total: 2.84s	remaining: 2.37s
545:	learn: 13.3884570	total: 2.85s	remaining: 2.37s
546:	learn: 13.3865032	total: 2.85s	remaining: 2.36s
547:	learn: 13.3858147	total: 2.86s	remaining: 2.36s
548:	learn: 13.3824765	total: 2.86s	remaining: 2.35s
549:	learn: 13.3785997	total: 2.87s	remaining: 2.35s
550:	learn: 13.3765860	total: 2.88s	remaining: 2.34s
551:	learn: 13.3695695	total: 2.88s	remaining: 2.34s
552:	learn: 13.3602165	total: 2.88s	remaining: 2.33s
553:	learn: 13.3587920	total: 2.89s	remaining: 2.33s
554:	learn: 13.3559480	total: 2.9s	remaining: 2.32s
555:	learn: 13.3480311	total: 2.9s	remaining: 2.32s
556:	learn: 13.3400041	total: 2.9s	remaining: 2.31s
557:	learn: 13.3388954	total: 2.91s	remaining: 2.31s
558:	learn: 13.3320706	total: 2.92s	remaining: 2.3s
559:	learn: 13.3217091	total: 2.92s	remaining: 2.2

697:	learn: 12.8290583	total: 3.64s	remaining: 1.57s
698:	learn: 12.8263957	total: 3.64s	remaining: 1.57s
699:	learn: 12.8255200	total: 3.65s	remaining: 1.56s
700:	learn: 12.8232088	total: 3.65s	remaining: 1.56s
701:	learn: 12.8189452	total: 3.66s	remaining: 1.55s
702:	learn: 12.8179491	total: 3.66s	remaining: 1.55s
703:	learn: 12.8170533	total: 3.67s	remaining: 1.54s
704:	learn: 12.8149156	total: 3.67s	remaining: 1.54s
705:	learn: 12.8088699	total: 3.68s	remaining: 1.53s
706:	learn: 12.8073647	total: 3.68s	remaining: 1.53s
707:	learn: 12.8020469	total: 3.69s	remaining: 1.52s
708:	learn: 12.7914514	total: 3.69s	remaining: 1.52s
709:	learn: 12.7901673	total: 3.7s	remaining: 1.51s
710:	learn: 12.7872918	total: 3.71s	remaining: 1.51s
711:	learn: 12.7861567	total: 3.71s	remaining: 1.5s
712:	learn: 12.7799925	total: 3.71s	remaining: 1.5s
713:	learn: 12.7773964	total: 3.72s	remaining: 1.49s
714:	learn: 12.7746029	total: 3.73s	remaining: 1.48s
715:	learn: 12.7721733	total: 3.73s	remaining: 1.

891:	learn: 12.2186162	total: 4.64s	remaining: 562ms
892:	learn: 12.2180174	total: 4.65s	remaining: 557ms
893:	learn: 12.2170062	total: 4.65s	remaining: 552ms
894:	learn: 12.2147924	total: 4.66s	remaining: 547ms
895:	learn: 12.2091209	total: 4.66s	remaining: 541ms
896:	learn: 12.2076717	total: 4.67s	remaining: 536ms
897:	learn: 12.2065687	total: 4.67s	remaining: 531ms
898:	learn: 12.2033368	total: 4.68s	remaining: 526ms
899:	learn: 12.1968864	total: 4.69s	remaining: 521ms
900:	learn: 12.1959886	total: 4.69s	remaining: 516ms
901:	learn: 12.1917554	total: 4.7s	remaining: 510ms
902:	learn: 12.1851973	total: 4.7s	remaining: 505ms
903:	learn: 12.1837924	total: 4.71s	remaining: 500ms
904:	learn: 12.1829596	total: 4.72s	remaining: 495ms
905:	learn: 12.1823426	total: 4.72s	remaining: 490ms
906:	learn: 12.1812653	total: 4.73s	remaining: 485ms
907:	learn: 12.1762739	total: 4.73s	remaining: 480ms
908:	learn: 12.1755506	total: 4.74s	remaining: 475ms
909:	learn: 12.1744186	total: 4.75s	remaining: 4

In [9]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from joblib import dump

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Calculate the index to split files into training and testing sets
split_index = int(len(csv_files) * 0.7)

# Training set and Testing set
train_files = csv_files[:split_index]
test_files = csv_files[split_index:]

# Initialize common columns with columns from the first CSV file
common_columns = set(pd.read_csv(os.path.join(folder_path, csv_files[0])).columns)

# Loop through each CSV file in the folder to find common columns
for csv_file in csv_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

# Print the common variables found in all files
print("\nCommon Variables:")
print(common_columns)

# Initialize an empty set to store all unique columns
all_columns = set()

# Loop through each CSV file in the folder to find all unique columns
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

# Find the uncommon variables
uncommon_variables = all_columns - common_columns

# Print the uncommon variables
print("\nUncommon Variables:")
print(uncommon_variables)

# Initialize an empty list to store the files in memory
files_in_memory = []

# Initialize the model
model = Lasso(alpha=0.1, max_iter=10000, random_state=42)

# Loop through each CSV file in the folder
for idx, csv_file in enumerate(csv_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]
    
    # Concatenate the data to the DataFrame
    if idx == 0:
        concatenated_data = df
    else:
        concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
        y = y.values.reshape(-1)
        y = y[:len(X)]
        #y = y.iloc[:, 0]

    # Fit the model on the new data
    model.fit(X, y)
    
    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")
    
    # Update files in memory
    files_in_memory.append(csv_file)

# Print all files in memory
print(f"All files in memory: {files_in_memory}")

# Extract features (X) and target (y) from concatenated data for test set
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

# Ensure y contains only one column
if isinstance(y_test, pd.DataFrame):
    y_test = y_test.values.reshape(-1)
    y_test = y_test[:len(X_test)]

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# Save the trained model as a .pkl file
model_filename = 'Lasso_model.pkl'
dump(model, model_filename)
print(f"Model saved as {model_filename}")



Common Variables:
{'1715 IMG_TIR1', '0015 IMG_TIR1', '0445 IMG_WV', '1115 IMG_TIR1', '1845 IMG_WV', '0345 IMG_WV', '1445 IMG_TIR1', '0615 IMG_WV', '0315 IMG_TIR1', '0945 IMG_TIR1', '1945 IMG_TIR2', '2315 IMG_TIR2', '0045 IMG_TIR2', '2215 IMG_WV', '1645 IMG_TIR1', '1545 IMG_TIR2', '2115 IMG_TIR1', '2145 IMG_TIR1', '1515 IMG_TIR2', '2345 IMG_TIR1', '1415 IMG_TIR1', '1915 IMG_WV', 'Date', '2345 IMG_TIR2', '0315 IMG_TIR2', '0645 IMG_TIR1', '0945 IMG_TIR2', '2045 IMG_TIR2', '2115 IMG_WV', '0345 IMG_TIR2', '1015 IMG_WV', '0445 IMG_TIR1', '2045 IMG_WV', '0715 IMG_WV', '1245 IMG_TIR2', '1245 IMG_WV', '1715 IMG_TIR2', '1315 IMG_TIR1', '1915 IMG_TIR2', '1945 IMG_TIR1', '1115 IMG_TIR2', '0945 IMG_WV', '2045 IMG_TIR1', '2215 IMG_TIR2', '0715 IMG_TIR2', '1815 IMG_TIR1', '1315 IMG_WV', '1015 IMG_TIR1', '0245 IMG_TIR1', '1745 IMG_TIR1', '2145 IMG_TIR2', '0615 IMG_TIR1', '2315 IMG_WV', '1345 IMG_TIR2', '1815 IMG_WV', '1045 IMG_WV', '0745 IMG_WV', '0245 IMG_TIR2', '1815 IMG_TIR2', 'longitude', '1515 I

/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.508e+07, tolerance: 4.834e+03
  model = cd_fast.enet_coordinate_descent(


Old files: []
New file: interpolated_insat_on_imerg_20180105.csv


/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.970e+06, tolerance: 7.978e+02
  model = cd_fast.enet_coordinate_descent(


Old files: ['interpolated_insat_on_imerg_20180105.csv']
New file: interpolated_insat_on_imerg_20180108.csv


...

/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.608e+07, tolerance: 4.108e+03
  model = cd_fast.enet_coordinate_descent(


Old files: ['interpolated_insat_on_imerg_20180105.csv', 'interpolated_insat_on_imerg_20180108.csv', 'interpolated_insat_on_imerg_20180103.csv', 'interpolated_insat_on_imerg_20180110.csv', 'interpolated_insat_on_imerg_20180109.csv', 'interpolated_insat_on_imerg_20180106.csv', 'interpolated_insat_on_imerg_20180107.csv']
New file: interpolated_insat_on_imerg_20180104.csv


/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.235e+07, tolerance: 3.497e+03
  model = cd_fast.enet_coordinate_descent(


Old files: ['interpolated_insat_on_imerg_20180105.csv', 'interpolated_insat_on_imerg_20180108.csv', 'interpolated_insat_on_imerg_20180103.csv', 'interpolated_insat_on_imerg_20180110.csv', 'interpolated_insat_on_imerg_20180109.csv', 'interpolated_insat_on_imerg_20180106.csv', 'interpolated_insat_on_imerg_20180107.csv', 'interpolated_insat_on_imerg_20180104.csv']
New file: interpolated_insat_on_imerg_20180101.csv
Old files: ['interpolated_insat_on_imerg_20180105.csv', 'interpolated_insat_on_imerg_20180108.csv', 'interpolated_insat_on_imerg_20180103.csv', 'interpolated_insat_on_imerg_20180110.csv', 'interpolated_insat_on_imerg_20180109.csv', 'interpolated_insat_on_imerg_20180106.csv', 'interpolated_insat_on_imerg_20180107.csv', 'interpolated_insat_on_imerg_20180104.csv', 'interpolated_insat_on_imerg_20180101.csv']
New file: interpolated_insat_on_imerg_20180102.csv
All files in memory: ['interpolated_insat_on_imerg_20180105.csv', 'interpolated_insat_on_imerg_20180108.csv', 'interpolated_in

/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.703e+06, tolerance: 2.539e+03
  model = cd_fast.enet_coordinate_descent(


In [25]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Calculate the index to split files into training and testing sets
split_index = int(len(csv_files) * 0.7)

# Training set and Testing set
train_files = csv_files[:split_index]
test_files = csv_files[split_index:]

# Initialize common columns with columns from the first CSV file
common_columns = set(pd.read_csv(os.path.join(folder_path, csv_files[0])).columns)

# Loop through each CSV file in the folder to find common columns
for csv_file in csv_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

# Print the common variables found in all files
print("\nCommon Variables:")
print(common_columns)

# Initialize an empty set to store all unique columns
all_columns = set()

# Loop through each CSV file in the folder to find all unique columns
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

# Find the uncommon variables
uncommon_variables = all_columns - common_columns

# Print the uncommon variables
print("\nUncommon Variables:")
print(uncommon_variables)

# Initialize an empty list to store the files in memory
files_in_memory = []

input_shape = X.shape[1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Initialize the model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(input_shape)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Loop through each CSV file in the folder
for idx, csv_file in enumerate(csv_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]
    
    # Concatenate the data to the DataFrame
    if idx == 0:
        concatenated_data = df
    else:
        concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
        y = y.values.reshape(-1)
        y = y[:len(X)]
        #y = y.iloc[:, 0]

    # Reshape X for Conv1D input
    X = X.values.reshape(X.shape[0], X.shape[1], 1)

    # Fit the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    
    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")
    
    # Update files in memory
    files_in_memory.append(csv_file)

# Print all files in memory
print(f"All files in memory: {files_in_memory}")

# Extract features (X) and target (y) from concatenated data for test set
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
    y_test = y_test.values.reshape(-1)
    y_test = y_test[:len(X_test)]


# Reshape X_test for Conv1D input
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# Save the trained model as a .h5 file
model_filename = 'CNN_model.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")



Common Variables:
{'1715 IMG_TIR1', '0015 IMG_TIR1', '0445 IMG_WV', '1115 IMG_TIR1', '1845 IMG_WV', '0345 IMG_WV', '1445 IMG_TIR1', '0615 IMG_WV', '0315 IMG_TIR1', '0945 IMG_TIR1', '1945 IMG_TIR2', '2315 IMG_TIR2', '0045 IMG_TIR2', '2215 IMG_WV', '1645 IMG_TIR1', '1545 IMG_TIR2', '2115 IMG_TIR1', '2145 IMG_TIR1', '1515 IMG_TIR2', '2345 IMG_TIR1', '1415 IMG_TIR1', '1915 IMG_WV', 'Date', '2345 IMG_TIR2', '0315 IMG_TIR2', '0645 IMG_TIR1', '0945 IMG_TIR2', '2045 IMG_TIR2', '2115 IMG_WV', '0345 IMG_TIR2', '1015 IMG_WV', '0445 IMG_TIR1', '2045 IMG_WV', '0715 IMG_WV', '1245 IMG_TIR2', '1245 IMG_WV', '1715 IMG_TIR2', '1315 IMG_TIR1', '1915 IMG_TIR2', '1945 IMG_TIR1', '1115 IMG_TIR2', '0945 IMG_WV', '2045 IMG_TIR1', '2215 IMG_TIR2', '0715 IMG_TIR2', '1815 IMG_TIR1', '1315 IMG_WV', '1015 IMG_TIR1', '0245 IMG_TIR1', '1745 IMG_TIR1', '2145 IMG_TIR2', '0615 IMG_TIR1', '2315 IMG_WV', '1345 IMG_TIR2', '1815 IMG_WV', '1045 IMG_WV', '0745 IMG_WV', '0245 IMG_TIR2', '1815 IMG_TIR2', 'longitude', '1515 I

625/625 [==============================] - 1s 2ms/step - loss: 6258.1528 - val_loss: 13568.9473
Epoch 9/10
625/625 [==============================] - 1s 2ms/step - loss: 5585819.0000 - val_loss: 96608496.0000
Epoch 10/10
625/625 [==============================] - 1s 2ms/step - loss: 32019536.0000 - val_loss: 1699446.5000
Old files: ['interpolated_insat_on_imerg_20180101.csv', 'interpolated_insat_on_imerg_20180102.csv', 'interpolated_insat_on_imerg_20180104.csv', 'interpolated_insat_on_imerg_20180106.csv']
New file: interpolated_insat_on_imerg_20180105.csv
Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 51605296.0000 - val_loss: 304127072.0000
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 992358912.0000 - val_loss: 504.2226
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 523.6475 - val_loss: 492.0539
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 607.3538 - val_loss: 478.0728
Epoch

/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Calculate the index to split files into training and testing sets
split_index = int(len(csv_files) * 0.7)

# Training set and Testing set
train_files = csv_files[:split_index]
test_files = csv_files[split_index:]

# Initialize common columns with columns from the first CSV file
common_columns = set(pd.read_csv(os.path.join(folder_path, csv_files[0])).columns)

# Loop through each CSV file in the folder to find common columns
for csv_file in csv_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

# Print the common variables found in all files
print("\nCommon Variables:")
print(common_columns)

# Initialize an empty set to store all unique columns
all_columns = set()

# Loop through each CSV file in the folder to find all unique columns
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

# Find the uncommon variables
uncommon_variables = all_columns - common_columns

# Print the uncommon variables
print("\nUncommon Variables:")
print(uncommon_variables)

# Initialize an empty list to store the files in memory
files_in_memory = []

# Loop through each CSV file in the folder
for idx, csv_file in enumerate(csv_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]
    
    # Concatenate the data to the DataFrame
    if idx == 0:
        concatenated_data = df
    else:
        concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
        y = y.values.reshape(-1)
        y = y[:len(X)]

    # Reshape X for LSTM input
    X = X.values.reshape(X.shape[0], 1, X.shape[1])

    # Initialize the model
    model = Sequential()
    model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    
    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")
    
    # Update files in memory
    files_in_memory.append(csv_file)

# Print all files in memory
print(f"All files in memory: {files_in_memory}")

# Extract features (X) and target (y) from concatenated data for test set
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
    y_test = y_test.values.reshape(-1)
    y_test = y_test[:len(X_test)]

# Reshape X_test for LSTM input
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# Save the trained model as a .h5 file
model_filename = 'RNN_model.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")



Common Variables:
{'1615 IMG_TIR2', '1545 IMG_TIR1', '2145 IMG_WV', '1345 IMG_TIR2', '0745 IMG_TIR2', '0615 IMG_TIR1', '0945 IMG_TIR2', '1515 IMG_TIR2', '2345 IMG_WV', '0245 IMG_TIR2', '0715 IMG_TIR1', '0945 IMG_TIR1', '1345 IMG_WV', '0215 IMG_TIR1', '0645 IMG_WV', '1415 IMG_TIR2', '0215 IMG_WV', '0015 IMG_TIR1', '2215 IMG_WV', '1445 IMG_TIR2', '1045 IMG_WV', '1615 IMG_TIR1', '1415 IMG_TIR1', 'latitude', '1845 IMG_TIR1', '1315 IMG_WV', '0215 IMG_TIR2', '0245 IMG_TIR1', '0715 IMG_TIR2', '1815 IMG_TIR2', '2115 IMG_TIR2', '1745 IMG_TIR1', '0115 IMG_TIR1', '0145 IMG_TIR1', '1045 IMG_TIR1', '2045 IMG_TIR2', '1945 IMG_WV', '0245 IMG_WV', '1145 IMG_WV', '1845 IMG_WV', '0545 IMG_WV', '1715 IMG_TIR1', '0745 IMG_WV', '0015 IMG_TIR2', '1515 IMG_TIR1', '0045 IMG_TIR1', '1715 IMG_WV', '1115 IMG_TIR2', '1315 IMG_TIR2', '0545 IMG_TIR2', '1845 IMG_TIR2', '2145 IMG_TIR2', '2315 IMG_TIR2', '1745 IMG_TIR2', '1415 IMG_WV', '0345 IMG_WV', '1815 IMG_TIR1', '2345 IMG_TIR2', 'precipitationCal', '0745 IMG_TIR

/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import math

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Calculate the index to split files into training and testing sets
split_index = int(len(csv_files) * 0.7)

# Training set and Testing set
train_files = csv_files[:split_index]
test_files = csv_files[split_index:]

# Initialize common columns with columns from the first CSV file
common_columns = set(pd.read_csv(os.path.join(folder_path, csv_files[0])).columns)

# Loop through each CSV file in the folder to find common columns
for csv_file in csv_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)
    
# Print the common variables found in all files
print("\nCommon Variables:")
print(common_columns)

# Initialize an empty set to store all unique columns
all_columns = set()

# Loop through each CSV file in the folder to find all unique columns
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

# Find the uncommon variables
uncommon_variables = all_columns - common_columns

# Print the uncommon variables
print("\nUncommon Variables:")
print(uncommon_variables)

# Initialize an empty list to store the files in memory
files_in_memory = []

# Loop through each CSV file in the folder
for idx, csv_file in enumerate(csv_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]
    
    # Concatenate the data to the DataFrame
    if idx == 0:
        concatenated_data = df
    else:
        concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
        y = y.values.reshape(-1)
        y = y[:len(X)]

    # Reshape X for LSTM input
    X = X.values.reshape(X.shape[0], 1, X.shape[1])

    # Initialize the model
    model = Sequential()
    model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    
    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")
    
    # Update files in memory
    files_in_memory.append(csv_file)

# Print all files in memory
print(f"All files in memory: {files_in_memory}")

# Extract features (X) and target (y) from concatenated data for test set
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
    y_test = y_test.values.reshape(-1)
    y_test = y_test[:len(X_test)]

# Reshape X_test for LSTM input
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mse)
r_squared = r2_score(y_test, y_pred)

print(f"Mean Squared Error on Test Set: {mse:.4f}")
print(f"Mean Absolute Error on Test Set: {mae:.4f}")
print(f"Root Mean Squared Error on Test Set: {rmse:.4f}")
print(f"R-squared on Test Set: {r_squared:.4f}")

# Save the trained model as a .h5 file
model_filename = 'RNN_model.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")



Common Variables:
{'0045 IMG_WV', '2015 IMG_TIR1', '1745 IMG_TIR2', '1945 IMG_TIR2', '1615 IMG_TIR1', '2245 IMG_WV', '2245 IMG_TIR1', '1845 IMG_TIR2', '0945 IMG_TIR1', '2145 IMG_WV', 'latitude', '1145 IMG_TIR2', '1545 IMG_TIR2', '1715 IMG_WV', '1045 IMG_TIR1', '0015 IMG_TIR1', '1845 IMG_WV', '1045 IMG_TIR2', '0545 IMG_TIR2', '1745 IMG_WV', '0215 IMG_TIR1', '0445 IMG_TIR1', '1145 IMG_WV', '0545 IMG_TIR1', '0015 IMG_WV', '1345 IMG_WV', '0315 IMG_WV', '1545 IMG_TIR1', '0215 IMG_WV', '1145 IMG_TIR1', '1545 IMG_WV', '1115 IMG_WV', '2045 IMG_TIR1', '1315 IMG_WV', '0115 IMG_TIR1', '2045 IMG_WV', '0615 IMG_TIR1', '2315 IMG_WV', '1245 IMG_TIR1', '2215 IMG_WV', '0615 IMG_TIR2', '0045 IMG_TIR1', '1815 IMG_TIR1', '1815 IMG_TIR2', '0545 IMG_WV', '1515 IMG_TIR1', '2115 IMG_TIR1', '1615 IMG_TIR2', '0715 IMG_WV', '0615 IMG_WV', '2345 IMG_TIR1', '1245 IMG_WV', '0315 IMG_TIR1', '0445 IMG_WV', '0745 IMG_TIR1', '1345 IMG_TIR1', '1645 IMG_TIR1', '1915 IMG_TIR1', '1415 IMG_TIR1', '0215 IMG_TIR2', '0115 IMG

/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2050/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Batch size
batch_size = 10

batch_wise_results = []

# Loop through files in batches
for start_index in range(0, len(csv_files), batch_size):
  # Get current batch of files
  batch_files = csv_files[start_index:start_index + batch_size]

  # Initialize common columns
  common_columns = set(pd.read_csv(os.path.join(folder_path, batch_files[0])).columns)

  # Find common columns within the batch
  for csv_file in batch_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

  # Print common variables for this batch
  print(f"\nCommon Variables (Batch {start_index // batch_size + 1}):")
  print(common_columns)

  # Initialize all columns
  all_columns = set()

  # Find all unique columns within the batch
  for csv_file in batch_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

  # Find uncommon variables for this batch
  uncommon_variables = all_columns - common_columns
  print(f"\nUncommon Variables (Batch {start_index // batch_size + 1}):")
  print(uncommon_variables)

  # Process data within the batch
  concatenated_data = None
  for idx, csv_file in enumerate(batch_files):
    # Load data from CSV file
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
      y = y.values.reshape(-1)
      y = y[:len(X)]

    # Reshape X for LSTM input
    X = X.values.reshape(X.shape[0], 1, X.shape[1])

    # Initialize the model (can be moved outside the loop if desired for efficiency)
    model = Sequential()
    model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    
    # Calculate metrics (replace with your desired metrics)
    train_loss = model.history['loss'][-1]
    val_loss = model.history['val_loss'][-1]

    # Store metrics and batch number in a dictionary
    batch_results = {
        "batch_num": start_index // batch_size + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        # Add other metrics you want to track (e.g., accuracy, F1 score)
    }
    
    # Concatenate data for all iterations
    concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Concatenate data for test set if first iteration
    if idx == 0:
      concatenated_data = df.copy()
    else:
      concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

  # Print processed files
  print(f"\nProcessed files: {batch_files}")

# After processing all batches, prepare and evaluate the test set

# Extract features (X_test) and target (y_test) from concatenated data
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
  y_test = y_test.values.reshape(-1)
  y_test = y_test[:len(X_test)]

# Reshape X_test for LSTM input
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
percent_error = (rmse / np.mean(y_test)) * 100
print(f"Mean Squared Error on Test Set: {mse:.4f}")
print(f"Root Mean Squared Error on Test Set: {rmse:.4f}")
print(f"Percentage Error on Test Set: {percent_error:.2f}%")

# Save the trained model as a .h5 file
model_filename = 'RNN_model.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")



Common Variables (Batch 1):
{'1615 IMG_TIR2', '1545 IMG_TIR1', '2145 IMG_WV', '1345 IMG_TIR2', '0745 IMG_TIR2', '0615 IMG_TIR1', '0945 IMG_TIR2', '1515 IMG_TIR2', '2345 IMG_WV', '0245 IMG_TIR2', '0715 IMG_TIR1', '0945 IMG_TIR1', '1345 IMG_WV', '0215 IMG_TIR1', '0645 IMG_WV', '1415 IMG_TIR2', '0215 IMG_WV', '0015 IMG_TIR1', '2215 IMG_WV', '1445 IMG_TIR2', '1045 IMG_WV', '1615 IMG_TIR1', '1415 IMG_TIR1', 'latitude', '1845 IMG_TIR1', '1315 IMG_WV', '0215 IMG_TIR2', '0245 IMG_TIR1', '0715 IMG_TIR2', '1815 IMG_TIR2', '2115 IMG_TIR2', '1745 IMG_TIR1', '0115 IMG_TIR1', '0145 IMG_TIR1', '1045 IMG_TIR1', '2045 IMG_TIR2', '1945 IMG_WV', '0245 IMG_WV', '1145 IMG_WV', '1845 IMG_WV', '0545 IMG_WV', '1715 IMG_TIR1', '0745 IMG_WV', '0015 IMG_TIR2', '1515 IMG_TIR1', '0045 IMG_TIR1', '1715 IMG_WV', '1115 IMG_TIR2', '1315 IMG_TIR2', '0545 IMG_TIR2', '1845 IMG_TIR2', '2145 IMG_TIR2', '2315 IMG_TIR2', '1745 IMG_TIR2', '1415 IMG_WV', '0345 IMG_WV', '1815 IMG_TIR1', '2345 IMG_TIR2', 'precipitationCal', '07

TypeError: 'History' object is not subscriptable

In [13]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2050/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Batch size
batch_size = 10

# Embedding dimension for categorical variables
embedding_dim = 8

# Loop through files in batches
for start_index in range(0, len(csv_files), batch_size):
  # Get current batch of files
  batch_files = csv_files[start_index:start_index + batch_size]

  # Initialize common columns and categorical columns
  common_columns = set()
  categorical_cols = set()

  # Find common and categorical columns within the batch
  for csv_file in batch_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns.update(df.columns)
    for col in df.columns:
      if df[col].dtypes == 'object':  # Assuming categorical variables are strings
        categorical_cols.add(col)
    common_columns.remove('precipitationCal')  # Exclude target

  # Process data within the batch
  concatenated_data = None
  for idx, csv_file in enumerate(batch_files):
    # Load data from CSV file
    df = pd.read_csv(os.path.join(folder_path, csv_file))

    # Convert date to datetime (assuming 'date_col' is the date column)
    df['date'] = pd.to_datetime(df['Date'], unit='s')  # Adjust unit if needed
    # Extract features from date (year, month, day of week)
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.weekday

    # Drop original date column
    df = df.drop(columns=['date_col', 'date'])

    # Extract features (X) and target (y) from the data
    X = df[list(common_columns) + list(categorical_cols)]
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
      y = y.values.reshape(-1)
      y = y[:len(X)]

    # Prepare categorical features for embedding layer
    categorical_features = {col: len(df[col].unique()) for col in categorical_cols}
    for col in categorical_cols:
      X.loc[:, col] = X[col].astype('category').cat.codes

    # Reshape X for LSTM input
    X_numerical = X.select_dtypes(exclude='category').values.reshape(X.shape[0], 1, -1)
    X_categorical = {col: X[col].values.reshape(-1, 1) for col in categorical_cols}

    # Initialize the model
    model = Sequential()
    # Embedding layer for categorical features
    for col, num_categories in categorical_features.items():
      model.add(Embedding(num_categories, embedding_dim, input_length=1)(X_categorical[col]))
    # Concatenate numerical and embedded features
    model.add(Concatenate(axis=-1)([model.output] + [X_numerical]))
    model.add(LSTM(64))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    
    # Concatenate data for test set if first iteration
    if idx == 0:
      concatenated_data = df.copy()
    else:
      concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

  # Print processed files
  print(f"\nProcessed files: {batch_files}")
              

# After processing all batches, prepare and evaluate the test set

# Extract features (X_test) and target (y_test) from concatenated data
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
  y_test = y_test.values.reshape(-1)
  y_test = y_test[:len(X_test)]

# Reshape X_test for LSTM input
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
percent_error = (rmse / np.mean(y_test)) * 100
print(f"Mean Squared Error on Test Set: {mse:.4f}")
print(f"Root Mean Squared Error on Test Set: {rmse:.4f}")
print(f"Percentage Error on Test Set: {percent_error:.2f}%")

# Save the trained model as a .h5 file
model_filename = 'RNN_model.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")

KeyError: "['date_col'] not found in axis"

In [21]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

# Specify paths for new data and saved model
new_data_folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2019'
saved_model_path = '/Users/kunalpathak9826/RNN_model.h5'

# List CSV files in the new data directory
new_data_files = [f for f in os.listdir(new_data_folder_path) if f.endswith('.csv')]

batch_size = 10

# Process new data in batches
for start_index in range(0, len(new_data_files), batch_size):
  # Get current batch of files
  batch_files = new_data_files[start_index:start_index + batch_size]

  # Initialize common columns
  common_columns = set(pd.read_csv(os.path.join(new_data_folder_path, batch_files[0])).columns)

  # Find common columns within the batch
  for csv_file in batch_files[1:]:
    df = pd.read_csv(os.path.join(new_data_folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

  # Print common variables for this batch
  print(f"\nCommon Variables (Batch {start_index // batch_size + 1}):")
  print(common_columns)

  # Initialize all columns
  all_columns = set()

  # Find all unique columns within the batch
  for csv_file in batch_files:
    df = pd.read_csv(os.path.join(new_data_folder_path, csv_file))
    all_columns.update(df.columns)

  # Find uncommon variables for this batch
  uncommon_variables = all_columns - common_columns
  print(f"\nUncommon Variables (Batch {start_index // batch_size + 1}):")
  print(uncommon_variables)

  # Initialize an empty DataFrame to store concatenated data
  concatenated_data = pd.DataFrame()

  # Process data within the batch
  for idx, csv_file in enumerate(batch_files):
        
    # Load the saved model within each batch iteration
    model = load_model(saved_model_path)
    
    # Load data from CSV file
    df = pd.read_csv(os.path.join(new_data_folder_path, csv_file))[list(common_columns) + ['precipitationCal']]

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
      y = y.values.reshape(-1)
      y = y[:len(X)]

    # Reshape X for LSTM input
    X = X.values.reshape(X.shape[0], 1, X.shape[1])

    # Continue training the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

    # Concatenate data for test set
    concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Print processed files
    print(f"\nProcessed files: {batch_files}")

# After processing all batches, prepare and evaluate the test set

# Extract features (X_test) and target (y_test) from concatenated data
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
  y_test = y_test.values.reshape(-1)
  y_test = y_test[:len(X_test)]

# Reshape X_test for LSTM input
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
percent_error = (rmse / np.mean(y_test)) * 100
print(f"Mean Squared Error on Test Set: {mse:.4f}")
print(f"Root Mean Squared Error on Test Set: {rmse:.4f}")
print(f"Percentage Error on Test Set: {percent_error:.2f}%")

# Save the trained model as a .h5 file
model_filename = 'RNN_model2.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")


Common Variables (Batch 1):
{'0515 IMG_TIR2', '2315 IMG_TIR1', '0245 IMG_WV', '2315 IMG_WV', '0245 IMG_TIR2', '0715 IMG_TIR1', '0515 IMG_TIR1', '1315 IMG_TIR1', '1645 IMG_TIR1', 'Date', '1315 IMG_TIR2', 'longitude', '2315 IMG_TIR2', '0515 IMG_WV', 'latitude', 'precipitationCal', '1315 IMG_WV', '0245 IMG_TIR1', '0715 IMG_TIR2', '0715 IMG_WV', '1645 IMG_TIR2', '1645 IMG_WV'}

Uncommon Variables (Batch 1):
{'1615 IMG_TIR2', '1545 IMG_TIR1', '2145 IMG_WV', '1345 IMG_TIR2', '0745 IMG_TIR2', '0615 IMG_TIR1', '0945 IMG_TIR2', '1515 IMG_TIR2', '0415 IMG_TIR1', '2345 IMG_WV', '0945 IMG_TIR1', '1345 IMG_WV', '0215 IMG_TIR1', '0645 IMG_WV', '1415 IMG_TIR2', '0215 IMG_WV', '0015 IMG_TIR1', '1215 IMG_TIR1', '2215 IMG_WV', '1445 IMG_TIR2', '1045 IMG_WV', '1615 IMG_TIR1', '1415 IMG_TIR1', '1845 IMG_TIR1', '0215 IMG_TIR2', '0815 IMG_TIR2', '1815 IMG_TIR2', '2115 IMG_TIR2', '1745 IMG_TIR1', '0115 IMG_TIR1', '0145 IMG_TIR1', '0845 IMG_WV', '1045 IMG_TIR1', '2045 IMG_TIR2', '1945 IMG_WV', '0415 IMG_TIR2

ValueError: in user code:

    File "/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Users/kunalpathak9826/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/kunalpathak9826/anaconda3/lib/python3.11/

In [26]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2050/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Batch size
batch_size = 10

# Function for timestamp comparison
def compare_timestamps(var1, var2):
  try:
    timestamp1 = int(var1.split("_")[0])
    timestamp2 = int(var2.split("_")[0])
  except ValueError:
    return False
  threshold = 1  # You can adjust this threshold as needed
  return abs(timestamp1 - timestamp2) <= threshold

batch_wise_results = []

# Loop through files in batches
for start_index in range(0, len(csv_files), batch_size):
  # Get current batch of files
  batch_files = csv_files[start_index:start_index + batch_size]

  # Find common columns within the batch considering nearest timestamps
  for csv_file in batch_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    current_columns = set(df.columns)

    # Update common columns based on nearest timestamps
    for col1 in common_columns.copy():
      for col2 in current_columns:
        try:
          timestamp1 = int(col1.split("_")[0])
          timestamp2 = int(col2.split("_")[0])
        except ValueError:
          continue
        if abs(timestamp1 - timestamp2) <= threshold:
          common_columns.add(col2)  # Add nearest timestamp variable as common
          common_columns.remove(col1)  # Remove original if a closer match is found

  # Print common variables for this batch
  print(f"\nCommon Variables (Batch {start_index // batch_size + 1}):")
  print(common_columns)

  # Process data within the batch
  concatenated_data = None
  for idx, csv_file in enumerate(batch_files):
    # Load data from CSV file
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]
    
    # Select only common columns and precipitationCal (drop uncommon columns)
    df = df[common_columns.union({'precipitationCal'})]

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
      y = y.values.reshape(-1)
      y = y[:len(X)]

    # Reshape X for LSTM input
    X = X.values.reshape(X.shape[0], 1, X.shape[1])

    # Initialize the model (can be moved outside the loop if desired for efficiency)
    model = Sequential()
    model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model on the new data
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    
    # Calculate metrics (replace with your desired metrics)
    train_loss = model.history['loss'][-1]
    val_loss = model.history['val_loss'][-1]

    # Store metrics and batch number in a dictionary
    batch_results = {
        "batch_num": start_index // batch_size + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        # Add other metrics you want to track (e.g., accuracy, F1 score)
    }
    
    # Concatenate data for all iterations
    concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Concatenate data for test set if first iteration
    if idx == 0:
      concatenated_data = df.copy()
    else:
      concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

  # Print processed files
  print(f"\nProcessed files: {batch_files}")

# After processing all batches, prepare and evaluate the test set

# Extract features (X_test) and target (y_test) from concatenated data
X_test = concatenated_data.drop(columns=['precipitationCal'])
y_test = concatenated_data['precipitationCal']

if isinstance(y_test, pd.DataFrame):
  y_test = y_test.values.reshape(-1)
  y_test = y_test[:len(X_test)]

# Reshape X_test for LSTM input
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
percent_error = (rmse / np.mean(y_test)) * 100
print(f"Mean Squared Error on Test Set: {mse:.4f}")
print(f"Root Mean Squared Error on Test Set: {rmse:.4f}")
print(f"Percentage Error on Test Set: {percent_error:.2f}%")

# Save the trained model as a .h5 file
model_filename = 'RNN_model.h5'
model.save(model_filename)
print(f"Model saved as {model_filename}")



Common Variables (Batch 1):
{'1615 IMG_TIR2', '1545 IMG_TIR1', '2145 IMG_WV', '1345 IMG_TIR2', '0745 IMG_TIR2', '0615 IMG_TIR1', '0945 IMG_TIR2', '1515 IMG_TIR2', '0415 IMG_TIR1', '2345 IMG_WV', '0245 IMG_TIR2', '0715 IMG_TIR1', '0945 IMG_TIR1', '1345 IMG_WV', '0215 IMG_TIR1', '0645 IMG_WV', '1415 IMG_TIR2', '0215 IMG_WV', '0015 IMG_TIR1', '1215 IMG_TIR1', '2215 IMG_WV', '1445 IMG_TIR2', '1045 IMG_WV', '1615 IMG_TIR1', '1415 IMG_TIR1', 'latitude', '1845 IMG_TIR1', '1315 IMG_WV', '0215 IMG_TIR2', '0815 IMG_TIR2', '0245 IMG_TIR1', '0715 IMG_TIR2', '1815 IMG_TIR2', '2115 IMG_TIR2', '1745 IMG_TIR1', '0115 IMG_TIR1', '0145 IMG_TIR1', '0845 IMG_WV', '1045 IMG_TIR1', '0515 IMG_TIR1', '2045 IMG_TIR2', '1945 IMG_WV', '0415 IMG_TIR2', '0245 IMG_WV', '1145 IMG_WV', '1845 IMG_WV', '0545 IMG_WV', '1715 IMG_TIR1', '0745 IMG_WV', '0015 IMG_TIR2', '1515 IMG_TIR1', '0045 IMG_TIR1', '1715 IMG_WV', '1115 IMG_TIR2', '1315 IMG_TIR2', '0545 IMG_TIR2', '1845 IMG_TIR2', '2145 IMG_TIR2', '2315 IMG_TIR2', '174

KeyError: "['0915 IMG_WV', '0915 IMG_TIR2', '0915 IMG_TIR1'] not in index"

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from joblib import dump

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Batch size
batch_size = 10

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Loop through batches of CSV files
for start_index in range(0, len(csv_files), batch_size):
  batch_files = csv_files[start_index:start_index + batch_size]

  # Initialize common columns with columns from the first file in the batch
  common_columns = set(pd.read_csv(os.path.join(folder_path, batch_files[0])).columns)

  # Loop through each CSV file in the batch to find common columns
  for csv_file in batch_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

  # Check if common variables are less than 5, skip the batch
  if len(common_columns) < 5:
    print(f"\nSkipping Batch {start_index // batch_size + 1} (Less than 5 common variables):")
    for file in batch_files:
      print(f"- {file}")
    continue

  # Print the common variables found in all files in this batch
  print(f"\nCommon Variables for Batch {start_index // batch_size + 1}:")
  print(common_columns)

  # Initialize an empty list to store all unique columns for this batch
  all_columns = set()

  # Loop through each CSV file in the batch to find all unique columns
  for csv_file in batch_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

  # Find the uncommon variables for this batch
  uncommon_variables = all_columns - common_columns

  # Print the uncommon variables for this batch
  print(f"\nUncommon Variables for Batch {start_index // batch_size + 1}:")
  print(uncommon_variables)

  # Initialize an empty list to store the files in memory for this batch
  files_in_memory = []

  # Initialize the model
  model = CatBoostRegressor(iterations=1000, learning_rate=0.01, random_state=42, verbose=True)

  # Loop through each CSV file in the batch
  for idx, csv_file in enumerate(batch_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]

    # Concatenate the data to the DataFrame
    if idx == 0:
      concatenated_data = df
    else:
      concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
      y = y.values.reshape(-1)
      y = y[:len(X)]

    # Fit the model on the new data
    model.fit(X, y)

    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")

    # Update files in memory for this batch
    files_in_memory.append(csv_file)

  # Print all files processed in this batch
  print(f"All files processed in Batch {start_index // batch_size + 1}: {files_in_memory}")

  # Extract features (X) and target (y) from concatenated data for test set
  X_test = concatenated_data.drop(columns=['precipitationCal'])
  y_test = concatenated_data['precipitationCal']

# Ensure y contains only one column
  if isinstance(y_test, pd.DataFrame):
        y_test = y_test.values.reshape(-1)
        y_test = y_test[:len(X_test)]
        
# Make predictions on the test set
  y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# Save the trained model as a .pkl file
model_filename = 'CatBoost_model.pkl'
dump(model, model_filename)
print(f"Model saved as {model_filename}")




Common Variables for Batch 1:
{'latitude', '1315 IMG_WV', '2015 IMG_TIR1', '1045 IMG_TIR2', '1015 IMG_WV', '1715 IMG_TIR1', '1515 IMG_TIR1', '0215 IMG_TIR2', '0645 IMG_WV', '0715 IMG_TIR1', '1545 IMG_WV', '0545 IMG_TIR1', '2045 IMG_TIR2', '1115 IMG_TIR1', '0615 IMG_TIR2', '1515 IMG_WV', '2115 IMG_WV', '2345 IMG_WV', '1245 IMG_TIR1', '2245 IMG_TIR1', '1115 IMG_TIR2', '1245 IMG_TIR2', '0045 IMG_TIR2', '1615 IMG_WV', '1415 IMG_WV', '0015 IMG_TIR1', '0145 IMG_TIR1', '2145 IMG_TIR1', '1145 IMG_WV', '0015 IMG_TIR2', '1445 IMG_TIR2', '0215 IMG_TIR1', '2045 IMG_TIR1', '2215 IMG_WV', '0015 IMG_WV', '2245 IMG_WV', '1345 IMG_WV', '1345 IMG_TIR2', '0115 IMG_TIR2', '1315 IMG_TIR1', '1915 IMG_TIR1', '1415 IMG_TIR1', '0745 IMG_WV', '1245 IMG_WV', '0145 IMG_WV', '1845 IMG_TIR1', '1045 IMG_WV', '1545 IMG_TIR1', '2145 IMG_TIR2', '2215 IMG_TIR1', '2315 IMG_WV', '2015 IMG_TIR2', '0045 IMG_TIR1', '2315 IMG_TIR1', '1145 IMG_TIR2', '2345 IMG_TIR2', '0045 IMG_WV', '2115 IMG_TIR1', '1945 IMG_TIR2', '0345 IMG_

113:	learn: 35.4812621	total: 3.1s	remaining: 24.1s
114:	learn: 35.4718350	total: 3.14s	remaining: 24.2s
115:	learn: 35.4647937	total: 3.16s	remaining: 24.1s
116:	learn: 35.4562652	total: 3.22s	remaining: 24.3s
117:	learn: 35.4486253	total: 3.26s	remaining: 24.3s
118:	learn: 35.4389162	total: 3.27s	remaining: 24.2s
119:	learn: 35.4320695	total: 3.29s	remaining: 24.1s
120:	learn: 35.4248647	total: 3.31s	remaining: 24.1s
121:	learn: 35.4189395	total: 3.33s	remaining: 24s
122:	learn: 35.4127199	total: 3.35s	remaining: 23.9s
123:	learn: 35.4042121	total: 3.38s	remaining: 23.9s
124:	learn: 35.3985013	total: 3.42s	remaining: 23.9s
125:	learn: 35.3934913	total: 3.45s	remaining: 23.9s
126:	learn: 35.3880973	total: 3.48s	remaining: 23.9s
127:	learn: 35.3813459	total: 3.5s	remaining: 23.8s
128:	learn: 35.3756996	total: 3.52s	remaining: 23.8s
129:	learn: 35.3705322	total: 3.55s	remaining: 23.8s
130:	learn: 35.3638717	total: 3.57s	remaining: 23.7s
131:	learn: 35.3575431	total: 3.6s	remaining: 23.7

274:	learn: 34.5731583	total: 7.42s	remaining: 19.6s
275:	learn: 34.5698598	total: 7.45s	remaining: 19.6s
276:	learn: 34.5665323	total: 7.48s	remaining: 19.5s
277:	learn: 34.5480935	total: 7.51s	remaining: 19.5s
278:	learn: 34.5441144	total: 7.54s	remaining: 19.5s
279:	learn: 34.5418957	total: 7.56s	remaining: 19.4s
280:	learn: 34.5392138	total: 7.58s	remaining: 19.4s
281:	learn: 34.5360823	total: 7.61s	remaining: 19.4s
282:	learn: 34.5332168	total: 7.64s	remaining: 19.4s
283:	learn: 34.5282019	total: 7.67s	remaining: 19.3s
284:	learn: 34.5253663	total: 7.69s	remaining: 19.3s
285:	learn: 34.5229168	total: 7.71s	remaining: 19.3s
286:	learn: 34.5098371	total: 7.73s	remaining: 19.2s
287:	learn: 34.5020524	total: 7.76s	remaining: 19.2s
288:	learn: 34.4974854	total: 7.8s	remaining: 19.2s
289:	learn: 34.4944057	total: 7.82s	remaining: 19.1s
290:	learn: 34.4918953	total: 7.85s	remaining: 19.1s
291:	learn: 34.4883746	total: 7.87s	remaining: 19.1s
292:	learn: 34.4854440	total: 7.9s	remaining: 1

435:	learn: 33.9732496	total: 11.7s	remaining: 15.2s
436:	learn: 33.9714741	total: 11.8s	remaining: 15.2s
437:	learn: 33.9684714	total: 11.8s	remaining: 15.1s
438:	learn: 33.9541371	total: 11.8s	remaining: 15.1s
439:	learn: 33.9459661	total: 11.9s	remaining: 15.1s
440:	learn: 33.9421904	total: 11.9s	remaining: 15.1s
441:	learn: 33.9404277	total: 11.9s	remaining: 15.1s
442:	learn: 33.9358638	total: 12s	remaining: 15s
443:	learn: 33.9338992	total: 12s	remaining: 15s
444:	learn: 33.9314681	total: 12s	remaining: 15s
445:	learn: 33.9301338	total: 12s	remaining: 15s
446:	learn: 33.9275467	total: 12.1s	remaining: 15s
447:	learn: 33.9255557	total: 12.2s	remaining: 15s
448:	learn: 33.9239300	total: 12.2s	remaining: 15s
449:	learn: 33.9228151	total: 12.2s	remaining: 14.9s
450:	learn: 33.9213669	total: 12.2s	remaining: 14.9s
451:	learn: 33.9176438	total: 12.3s	remaining: 14.9s
452:	learn: 33.9154628	total: 12.3s	remaining: 14.9s
453:	learn: 33.9131505	total: 12.4s	remaining: 14.9s
454:	learn: 33.

597:	learn: 33.4409809	total: 17s	remaining: 11.4s
598:	learn: 33.4382691	total: 17s	remaining: 11.4s
599:	learn: 33.4370809	total: 17s	remaining: 11.4s
600:	learn: 33.4275728	total: 17.1s	remaining: 11.3s
601:	learn: 33.4168709	total: 17.1s	remaining: 11.3s
602:	learn: 33.4152110	total: 17.2s	remaining: 11.3s
603:	learn: 33.4112133	total: 17.2s	remaining: 11.3s
604:	learn: 33.4085363	total: 17.2s	remaining: 11.3s
605:	learn: 33.4061016	total: 17.3s	remaining: 11.2s
606:	learn: 33.4044535	total: 17.3s	remaining: 11.2s
607:	learn: 33.3991601	total: 17.3s	remaining: 11.2s
608:	learn: 33.3958641	total: 17.3s	remaining: 11.1s
609:	learn: 33.3929816	total: 17.4s	remaining: 11.1s
610:	learn: 33.3917719	total: 17.4s	remaining: 11.1s
611:	learn: 33.3897923	total: 17.4s	remaining: 11s
612:	learn: 33.3875466	total: 17.4s	remaining: 11s
613:	learn: 33.3859050	total: 17.5s	remaining: 11s
614:	learn: 33.3839056	total: 17.5s	remaining: 10.9s
615:	learn: 33.3722720	total: 17.5s	remaining: 10.9s
616:	

...

357:	learn: 40.8676999	total: 9.83s	remaining: 17.6s
358:	learn: 40.8605184	total: 9.87s	remaining: 17.6s
359:	learn: 40.8564230	total: 9.95s	remaining: 17.7s
360:	learn: 40.8533324	total: 9.97s	remaining: 17.7s
361:	learn: 40.8483586	total: 10s	remaining: 17.7s
362:	learn: 40.8444399	total: 10.1s	remaining: 17.6s
363:	learn: 40.8411115	total: 10.1s	remaining: 17.6s
364:	learn: 40.8358719	total: 10.1s	remaining: 17.6s
365:	learn: 40.8239411	total: 10.1s	remaining: 17.5s
366:	learn: 40.8184871	total: 10.1s	remaining: 17.5s
367:	learn: 40.8150639	total: 10.2s	remaining: 17.5s
368:	learn: 40.8122239	total: 10.2s	remaining: 17.4s
369:	learn: 40.8076974	total: 10.2s	remaining: 17.4s
370:	learn: 40.8034341	total: 10.2s	remaining: 17.4s
371:	learn: 40.7992773	total: 10.3s	remaining: 17.4s
372:	learn: 40.7963562	total: 10.3s	remaining: 17.3s
373:	learn: 40.7928017	total: 10.4s	remaining: 17.3s
374:	learn: 40.7878976	total: 10.4s	remaining: 17.3s
375:	learn: 40.7849063	total: 10.4s	remaining: 1

518:	learn: 39.8433164	total: 14.8s	remaining: 13.7s
519:	learn: 39.8405561	total: 14.8s	remaining: 13.7s
520:	learn: 39.8378346	total: 14.8s	remaining: 13.6s
521:	learn: 39.8251079	total: 14.8s	remaining: 13.6s
522:	learn: 39.8225460	total: 14.9s	remaining: 13.6s
523:	learn: 39.8207105	total: 14.9s	remaining: 13.5s
524:	learn: 39.8183547	total: 14.9s	remaining: 13.5s
525:	learn: 39.8154166	total: 15s	remaining: 13.5s
526:	learn: 39.8129364	total: 15s	remaining: 13.5s
527:	learn: 39.8078515	total: 15s	remaining: 13.4s
528:	learn: 39.8050351	total: 15s	remaining: 13.4s
529:	learn: 39.8016325	total: 15.1s	remaining: 13.4s
530:	learn: 39.7884720	total: 15.1s	remaining: 13.4s
531:	learn: 39.7854165	total: 15.2s	remaining: 13.3s
532:	learn: 39.7836611	total: 15.2s	remaining: 13.3s
533:	learn: 39.7791322	total: 15.2s	remaining: 13.3s
534:	learn: 39.7766748	total: 15.3s	remaining: 13.3s
535:	learn: 39.7740704	total: 15.3s	remaining: 13.3s
536:	learn: 39.7582085	total: 15.3s	remaining: 13.2s
5

674:	learn: 38.9998071	total: 19.2s	remaining: 9.24s
675:	learn: 38.9962599	total: 19.2s	remaining: 9.21s
676:	learn: 38.9927579	total: 19.2s	remaining: 9.18s
677:	learn: 38.9839999	total: 19.3s	remaining: 9.16s
678:	learn: 38.9792172	total: 19.3s	remaining: 9.13s
679:	learn: 38.9759693	total: 19.3s	remaining: 9.1s
680:	learn: 38.9727830	total: 19.4s	remaining: 9.07s
681:	learn: 38.9684075	total: 19.4s	remaining: 9.05s
682:	learn: 38.9637851	total: 19.4s	remaining: 9.02s
683:	learn: 38.9594516	total: 19.5s	remaining: 8.99s
684:	learn: 38.9553734	total: 19.5s	remaining: 8.97s
685:	learn: 38.9532765	total: 19.5s	remaining: 8.93s
686:	learn: 38.9352690	total: 19.5s	remaining: 8.9s
687:	learn: 38.9324359	total: 19.6s	remaining: 8.87s
688:	learn: 38.9301832	total: 19.6s	remaining: 8.84s
689:	learn: 38.9277852	total: 19.6s	remaining: 8.81s
690:	learn: 38.9235785	total: 19.6s	remaining: 8.78s
691:	learn: 38.9200240	total: 19.6s	remaining: 8.74s
692:	learn: 38.9177756	total: 19.7s	remaining: 8

830:	learn: 38.2520806	total: 23.9s	remaining: 4.87s
831:	learn: 38.2438963	total: 23.9s	remaining: 4.83s
832:	learn: 38.2427941	total: 24s	remaining: 4.81s
833:	learn: 38.2409029	total: 24s	remaining: 4.78s
834:	learn: 38.2331207	total: 24s	remaining: 4.75s
835:	learn: 38.2235498	total: 24s	remaining: 4.71s
836:	learn: 38.2215274	total: 24.1s	remaining: 4.68s
837:	learn: 38.2201855	total: 24.1s	remaining: 4.66s
838:	learn: 38.2083448	total: 24.1s	remaining: 4.63s
839:	learn: 38.2069315	total: 24.2s	remaining: 4.6s
840:	learn: 38.1886899	total: 24.2s	remaining: 4.58s
841:	learn: 38.1797648	total: 24.2s	remaining: 4.55s
842:	learn: 38.1762595	total: 24.3s	remaining: 4.52s
843:	learn: 38.1736389	total: 24.3s	remaining: 4.49s
844:	learn: 38.1713510	total: 24.3s	remaining: 4.46s
845:	learn: 38.1672549	total: 24.3s	remaining: 4.43s
846:	learn: 38.1660794	total: 24.4s	remaining: 4.4s
847:	learn: 38.1642581	total: 24.4s	remaining: 4.37s
848:	learn: 38.1626429	total: 24.4s	remaining: 4.34s
849

991:	learn: 37.5232042	total: 28.7s	remaining: 232ms
992:	learn: 37.5208637	total: 28.8s	remaining: 203ms
993:	learn: 37.5180709	total: 28.8s	remaining: 174ms
994:	learn: 37.5150956	total: 28.9s	remaining: 145ms
995:	learn: 37.5112663	total: 28.9s	remaining: 116ms
996:	learn: 37.5087789	total: 28.9s	remaining: 87ms
997:	learn: 37.5072781	total: 28.9s	remaining: 58ms
998:	learn: 37.5055096	total: 29s	remaining: 29ms
999:	learn: 37.5026623	total: 29s	remaining: 0us
Old files: ['interpolated_insat_on_imerg_20180101.csv', 'interpolated_insat_on_imerg_20180102.csv', 'interpolated_insat_on_imerg_20180107.csv', 'interpolated_insat_on_imerg_20180110.csv', 'interpolated_insat_on_imerg_20180105.csv', 'interpolated_insat_on_imerg_20180104.csv', 'interpolated_insat_on_imerg_20180109.csv', 'interpolated_insat_on_imerg_20180108.csv', 'interpolated_insat_on_imerg_20180106.csv']
New file: interpolated_insat_on_imerg_20180103.csv
All files processed in Batch 1: ['interpolated_insat_on_imerg_20180101.cs

In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from joblib import dump

# Directory containing your CSV files
folder_path = '/Users/kunalpathak9826/Desktop/ISRO/Data/Interpolated CSV/2018/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Batch size
batch_size = 10

# Shuffle the list of CSV files
import random
random.shuffle(csv_files)

# Loop through batches of CSV files
for start_index in range(0, len(csv_files), batch_size):
  batch_files = csv_files[start_index:start_index + batch_size]

  # Initialize common columns with columns from the first file in the batch
  common_columns = set(pd.read_csv(os.path.join(folder_path, batch_files[0])).columns)

  # Loop through each CSV file in the batch to find common columns
  for csv_file in batch_files[1:]:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    common_columns = common_columns.intersection(df.columns)

  # Check if common variables are less than 5, skip the batch
  if len(common_columns) < 5:
    print(f"\nSkipping Batch {start_index // batch_size + 1} (Less than 5 common variables):")
    for file in batch_files:
      print(f"- {file}")
    continue

  # Print the common variables found in all files in this batch
  print(f"\nCommon Variables for Batch {start_index // batch_size + 1}:")
  print(common_columns)

  # Initialize an empty list to store all unique columns for this batch
  all_columns = set()

  # Loop through each CSV file in the batch to find all unique columns
  for csv_file in batch_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    all_columns.update(df.columns)

  # Find the uncommon variables for this batch
  uncommon_variables = all_columns - common_columns

  # Print the uncommon variables for this batch
  print(f"\nUncommon Variables for Batch {start_index // batch_size + 1}:")
  print(uncommon_variables)

  # Initialize an empty list to store the files in memory for this batch
  files_in_memory = []

  # Initialize the model
  model = XGBRegressor(random_state=42)

  # Loop through each CSV file in the batch
  for idx, csv_file in enumerate(batch_files):
    # Load the data from CSV file, keeping only the common columns
    df = pd.read_csv(os.path.join(folder_path, csv_file))[list(common_columns) + ['precipitationCal']]

    # Concatenate the data to the DataFrame
    if idx == 0:
      concatenated_data = df
    else:
      concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

    # Extract features (X) and target (y) from the data
    X = df.drop(columns=['precipitationCal'])
    y = df['precipitationCal']

    # Ensure y contains only one column
    if isinstance(y, pd.DataFrame):
      y = y.values.reshape(-1)
      y = y[:len(X)]

    # Fit the model on the new data
    model.fit(X, y)

    # Print old and new files
    print(f"Old files: {files_in_memory}")
    print(f"New file: {csv_file}")

    # Update files in memory for this batch
    files_in_memory.append(csv_file)

  # Print all files processed in this batch
  print(f"All files processed in Batch {start_index // batch_size + 1}: {files_in_memory}")

  # Extract features (X) and target (y) from concatenated data for test set
  X_test = concatenated_data.drop(columns=['precipitationCal'])
  y_test = concatenated_data['precipitationCal']

# Ensure y contains only one column
  if isinstance(y_test, pd.DataFrame):
        y_test = y_test.values.reshape(-1)
        y_test = y_test[:len(X_test)]
        
# Make predictions on the test set
  y_pred = model.predict(X_test)

# Calculate the mean squared error for the testing set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# Save the trained model as a .pkl file
model_filename = 'XGBoost_model.pkl'
dump(model, model_filename)
print(f"Model saved as {model_filename}")




Common Variables for Batch 1:
{'latitude', '1315 IMG_WV', '2015 IMG_TIR1', '1045 IMG_TIR2', '1015 IMG_WV', '1715 IMG_TIR1', '1515 IMG_TIR1', '0215 IMG_TIR2', '0645 IMG_WV', '0715 IMG_TIR1', '1545 IMG_WV', '0545 IMG_TIR1', '2045 IMG_TIR2', '1115 IMG_TIR1', '0615 IMG_TIR2', '1515 IMG_WV', '2115 IMG_WV', '2345 IMG_WV', '1245 IMG_TIR1', '2245 IMG_TIR1', '1115 IMG_TIR2', '1245 IMG_TIR2', '0045 IMG_TIR2', '1615 IMG_WV', '1415 IMG_WV', '0015 IMG_TIR1', '0145 IMG_TIR1', '2145 IMG_TIR1', '1145 IMG_WV', '0015 IMG_TIR2', '1445 IMG_TIR2', '0215 IMG_TIR1', '2045 IMG_TIR1', '2215 IMG_WV', '0015 IMG_WV', '2245 IMG_WV', '1345 IMG_WV', '1345 IMG_TIR2', '0115 IMG_TIR2', '1315 IMG_TIR1', '1915 IMG_TIR1', '1415 IMG_TIR1', '0745 IMG_WV', '1245 IMG_WV', '0145 IMG_WV', '1845 IMG_TIR1', '1045 IMG_WV', '1545 IMG_TIR1', '2145 IMG_TIR2', '2215 IMG_TIR1', '2315 IMG_WV', '2015 IMG_TIR2', '0045 IMG_TIR1', '2315 IMG_TIR1', '1145 IMG_TIR2', '2345 IMG_TIR2', '0045 IMG_WV', '2115 IMG_TIR1', '1945 IMG_TIR2', '0345 IMG_